**Make plots of monitoring elements**

In particular:
- Limited to making plots of a single 2D ME.
- Make plots of single lumisections, a selected range of lumisections (gif), all lumisections in a selected run (gif)

In [ ]:
### imports

# external modules
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import importlib

# internal modules
sys.path.append('../utils')
import dataframe_utils as dfu
import plot_utils as pu
import hist_utils as hu
importlib.reload(dfu)
importlib.reload(pu)
importlib.reload(hu)

In [ ]:
### read the data
# OPTION 1: from previously stored file

# todo

In [ ]:
### read the data
# OPTION 2: directly from DIALS

# imports and authentication
from cmsdials.auth.bearer import Credentials
from cmsdials import Dials
from cmsdials.filters import RunFilters
from cmsdials.filters import LumisectionHistogram2DFilters

creds = Credentials.from_creds_file()
dials = Dials(creds, workspace='tracker')

In [ ]:
### read the data
# OPTION 2, continued

# find out in which dataset the run of interest is located
dataset = '/ZeroBias/Run.*-PromptReco-v1/DQMIO'
run_number = 381364
filters = RunFilters(
    dataset__regex=dataset,
    run_number=run_number
)

data = dials.run.list_all(
    filters,
    enable_progress=True
)
df = data.to_pandas()
df.head()

In [ ]:
### read the data
# OPTION 2, continued

# make a filter
dataset = '/ZeroBias/Run2024E-PromptReco-v1/DQMIO'
#mename = 'PixelPhase1/Phase1_MechanicalView/PXBarrel/clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_1'
mename = 'PixelPhase1/Phase1_MechanicalView/PXForward/clusters_per_SignedDiskCoord_per_SignedBladePanelCoord_PXRing_2'
run_number = 381364
filters = LumisectionHistogram2DFilters(
    dataset=dataset,
    me=mename,
    run_number=run_number
)

data = dials.h2d.list_all(
    filters,
    enable_progress=True
)
df = data.to_pandas()
df.head()

In [ ]:
### print some dataframe properties

print('Number of runs: {}'.format(len(dfu.get_runs(df))))
print('Number of LS: {}'.format(len(dfu.get_ls(df))))
print('Avaialable MEs: {}'.format(dfu.get_menames(df)))

In [ ]:
### filter the data (optional)

# location of DCS-on json file
# note: make sure to modify or add the correct json files depending on your data
dcsjson = '/eos/user/c/cmsdqm/www/CAF/certification/Collisions24/DCSOnly_JSONS/dailyDCSOnlyJSON/Collisions24_13p6TeV_378981_381594_DCSOnly_TkPx.json'

# filter the data
# note: modify to your taste
dff = dfu.select_json(df, dcsjson)

print('Filtered number of runs: {}'.format(len(dfu.get_runs(dff))))
print('Filtered number of LS: {}'.format(len(dff)))
print('All available run numbers in selected data:\n{}'.format(dfu.get_runs(dff)))

In [ ]:
### settings for plot aesthetics

xaxtitle = 'Some x title'
yaxtitle = 'Some y title'
caxtitle = 'Some c title'
metext = 'Clusters'
extracmstext = 'Preliminary'
lumitext = '2024 (13.6 TeV)'
xmin = df['x_min'].values[0]
xmax = df['x_max'].values[0]
ymin = df['y_min'].values[0]
ymax = df['y_max'].values[0]
xbins = int(df['x_bin'].values[0])
ybins = int(df['y_bin'].values[0])

In [ ]:
### make plot of a single lumisection in a selected run

# settings
run = 381364
ls = 100

# get the histogram
dfr = dfu.select_runs(dff,[run])
dfr = dfu.select_ls(dfr,[ls])
if len(dfr)!=1:
    raise Exception('ERROR: expected one instance, found {}'.format(len(dfr)))
hist = np.array(dfr['data'][0])
print('Shape of hist array: {}'.format(hist.shape))

# plot the histogram
lstext = 'Run: {}, LS: {}'.format(run, ls)
fig,ax = pu.plot_hist_2d(hist, figsize=(7,7), 
                         xaxtitle=xaxtitle, xaxtitlesize=12,
                         yaxtitle=yaxtitle, yaxtitlesize=12,
                         caxtitle=caxtitle, caxtitlesize=15, caxtitleoffset=20)
ax.text(0.02, 1.02, lstext, transform=ax.transAxes, fontsize=13)
ax.text(0.98, 1.02, metext, transform=ax.transAxes, fontsize=13, ha='right')
plt.show()
#fig.savefig('run{}.png'.format(run), dpi=300)

In [ ]:
### make gif image of some selected lumisections (or all lumisections) in a selected run

# settings
run = 381364
ls = None # use either an iterable (e.g. list, range, array, etc.) or None (for all lumisections)

# get the histogram(s)
dfr = dfu.select_runs(dff,[run])
if ls is not None: dfr = dfu.select_ls(dfr,ls)
print('Selected number of LS: {}'.format(len(dfr)))
runs = np.array(dfr['run_number'].values)
lumis = np.array(dfr['ls_number'].values)
hists = np.transpose(np.dstack(dfr['data'].values), axes=[2,0,1])
print('Shape of hists array: {}'.format(hists.shape))

# set some plot aesthetics
titles = ['Clusters PXRing 2, Run {}, LS {}'.format(runs[i], lumis[i]) for i in range(len(hists))]
figname = 'temp_gif.gif'
#caxrange = (0.01,2000)
caxrange = None
duration = 300

# make the gif
pu.plot_hists_2d_gif(hists,
                     titles=titles,
                     figname=figname,
                     caxrange=caxrange,
                     duration=duration,
                     mode='imageio')

In [ ]:
import IPython
from IPython.display import Image
Image(filename=figname)